In [1]:
# タイムスタンプ + MACアドレスのデータを
# MACアドレス, in時間, out時間に変換する

In [2]:
import pandas as pd

In [18]:
# csvファイルの読み込み
df = pd.read_csv('csv/20190227_kanda_b.csv')

# df = pd.read_csv('csv/20190123_1200_kanda.csv', usecols = [2,4], skiprows = 1)
# df = pd.read_csv('csv/20190123_1200_kanda.csv', skiprows = 1)
# df = pd.read_csv('csv/20190123_1200_kanda.csv', skiprows = 2, header=None)
# df = pd.read_csv('csv/20190123_1200_kanda.csv', nrows = 50) # ちょっとだけ読み込む for debug

In [19]:
# 無効なデータが入っている1行目を削除
df = df.drop(0, axis=0)


In [20]:
# 日付をintに変換
df_int_cast = df.astype({'year': int, 'month': int, 'day': int, 'hour': int, 'minute': int, 'second': int})

In [21]:
# コピー
df_new = df_int_cast

In [22]:
df_int_cast

,year,month,day,hour,minute,second,address
1,2019,2,27,16,40,0,5D:C7:18:0A:09:99
2,2019,2,27,16,40,0,4C:46:6E:12:53:B5
3,2019,2,27,16,40,0,63:69:DE:04:74:26
4,2019,2,27,16,40,0,49:B0:89:31:C8:2D
5,2019,2,27,16,40,0,6D:58:80:B9:50:9E
6,2019,2,27,16,40,0,46:66:97:A7:75:C1
7,2019,2,27,16,40,0,5C:20:1E:9F:13:83
8,2019,2,27,16,40,0,42:48:B7:1E:AA:F0
9,2019,2,27,16,40,0,58:3D:CF:90:22:33
10,2019,2,27,16,40,0,72:FB:1F:42:27:4D


In [23]:
# 時刻を秒に変換
days = 0 #df_int_cast['year'] * 365 + df_int_cast['month'] * 12 + df_int_cast['day']
hours = days * 24 + df_int_cast['hour']
minutes = hours * 60 + df_int_cast['minute']
df_new['elapsed_time'] = minutes * 60 + df_int_cast['second'] 

In [24]:
# 型を表示
print(df_new.dtypes)

year             int32
month            int32
day              int32
hour             int32
minute           int32
second           int32
address         object
elapsed_time     int32
dtype: object


In [25]:
# 時刻の列を削除
df_address_time = df_new.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1)

In [26]:
# Bleデータを管理するクラス
# addressが同じで前回計測されてからpersistence以内であればout_timeを更新する
# そうでなければFalseを返し、外部で新規BLEデバイスとしてリストに追加する
class BleLog:
    def __init__(self, address, time, persistence):
        self.address = address
        self.in_time = time 
        self.out_time = time
        self.persistence = persistence
        
    def update(self, address, time):
        
        # check address
        if self.address != address:
            return False
        
        # check elapsed time
        if time - self.out_time > self.persistence:
            return False
        
        # update out time
        self.out_time = time
        
        return True
    
    def print_data(self):
        start_h = int(self.in_time / 3600)
        start_m = int((self.in_time - start_h * 3600) / 60)
        start_s = int(self.in_time - start_h * 3600 - start_m * 60)
        end_h = int(self.out_time / 3600)
        end_m = int((self.out_time - end_h * 3600) / 60)
        end_s = int(self.out_time - end_h * 3600 - end_m * 60)
#         print(self.address + " " + str(self.in_time) + " " + str(self.out_time) + " " + str(self.out_time - self.in_time))
        diff = self.out_time - self.in_time
        print('address: %s | start %02s:%02d:%02d -> end %02s:%02d:%02d | time: %d' %(self.address, start_h, start_m, start_s, end_h, end_m, end_s, diff))
        

In [27]:
# 現在のリストと比較する
def update_ble_data(address, time):
    for ble in ble_list:
        if ble.update(address, time):
            return True
    
    return False


In [28]:
# bleデータを表示する
def print_ble_data(data):
    for ble in data:
        ble.print_data()
    

In [29]:
# リストを初期化
ble_list = []

# csvからデータを読み込みbleリストと比較していく
# リストになければ新規にbleデバイスを追加
for index, row in df_address_time.iterrows():
    addr = row['address']
    time = row['elapsed_time']
    
    if not update_ble_data(addr, time):
        new_ble = BleLog(addr, time, 120)
        ble_list.append(new_ble)
        

In [30]:
len(ble_list)

743

In [31]:
# 結果を表示
print_ble_data(ble_list)

address: 5D:C7:18:0A:09:99 | start 16:40:00 -> end 16:42:47 | time: 167
address: 4C:46:6E:12:53:B5 | start 16:40:00 -> end 16:40:43 | time: 43
address: 63:69:DE:04:74:26 | start 16:40:00 -> end 16:40:47 | time: 47
address: 49:B0:89:31:C8:2D | start 16:40:00 -> end 16:40:45 | time: 45
address: 6D:58:80:B9:50:9E | start 16:40:00 -> end 16:40:14 | time: 14
address: 46:66:97:A7:75:C1 | start 16:40:00 -> end 16:46:26 | time: 386
address: 5C:20:1E:9F:13:83 | start 16:40:00 -> end 16:43:42 | time: 222
address: 42:48:B7:1E:AA:F0 | start 16:40:00 -> end 16:44:46 | time: 286
address: 58:3D:CF:90:22:33 | start 16:40:00 -> end 16:41:27 | time: 87
address: 72:FB:1F:42:27:4D | start 16:40:00 -> end 16:40:00 | time: 0
address: 65:D0:8A:86:CD:F7 | start 16:40:00 -> end 16:40:38 | time: 38
address: 2C:41:A1:57:39:4C | start 16:40:00 -> end 16:40:21 | time: 21
address: 46:30:3F:DB:0B:9B | start 16:40:00 -> end 16:40:13 | time: 13
address: 74:28:87:E1:4D:FA | start 16:40:00 -> end 16:50:59 | time: 659
ad